In [1]:
from pyspark.sql import SparkSession

# Credenciais e endpoint do MinIO
MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"

# 1. Configurar a SparkSession para se conectar ao MinIO
spark = (
    SparkSession.builder
    .appName("MinIO-Integration-Test")
    .master("spark://spark-master:7077")
    # Configurações para o conector S3A
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true") # Essencial para MinIO
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

print("Spark Session configurada para MinIO!")

# 2. Criar um DataFrame de exemplo
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3)]
columns = ["nome", "id"]
df = spark.createDataFrame(data, columns)

print("DataFrame de exemplo criado:")
df.show()

# 3. Escrever o DataFrame no MinIO em formato Parquet
# O caminho usa o padrão s3a://<bucket-name>/<path-to-file>
bucket_name = "database"
output_path = f"s3a://{bucket_name}/meus_dados_parquet"

print(f"Escrevendo dados em: {output_path}")
df.write.mode("overwrite").parquet(output_path)

print("Escrita concluída!")

# 4. Ler os dados de volta do MinIO
print(f"Lendo dados de: {output_path}")
df_lido = spark.read.parquet(output_path)

print("DataFrame lido do MinIO:")
df_lido.show()

# Pare a sessão para liberar recursos
spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/23 23:45:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session configurada para MinIO!
DataFrame de exemplo criado:


+-------+---+
|   nome| id|
+-------+---+
|  Alice|  1|
|    Bob|  2|
|Charlie|  3|
+-------+---+

Escrevendo dados em: s3a://database/meus_dados_parquet


25/10/23 23:46:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

Escrita concluída!
Lendo dados de: s3a://database/meus_dados_parquet
DataFrame lido do MinIO:
+-------+---+
|   nome| id|
+-------+---+
|    Bob|  2|
|Charlie|  3|
|  Alice|  1|
+-------+---+

